Time to dig into ezomero. It's always important to remember that `ezomero` is using the `omero-py library` under the hood, which in turn uses a framework named `Ice` to talk directly to the OMERO server, rather than going through the web interface. The beginning is familiar: import configs, `ezomero`, and matplotlib. 

In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT
import matplotlib.pyplot as plt
import ezomero

Again, pretty familiar: create a connection object.

In [ ]:
conn = ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)

There are a few extra options for `ezomero.connect` in case you don't want to pass all the parameters directly in your code; you can set environment variables such as `OMERO_USER` and `OMERO_PASS` and `ezomero` will use them automatically, and you can also have a hidden file named `.ezomero` with your connection parameters - those will also be used automatically. This function will first use any parameters passed directly to the function, then check environment variables, then the `.ezomero` file, and finally prompt the user directly for anything that is still missing.

Note that `ezomero` is fully compatible with the regular `omero-py` API - the object you get from `ezomero.connect` can be used by regular `omero-py` functions as well. As an example, here is some code to look at users and groups using the regular `omero-py` API, but re-using the `conn` object created by `ezomero`.

In [ ]:
user = conn.getUser()
print("user ID:", user.getId())

# Check if you are an Administrator
print("Is Admin:", conn.isAdmin())

print("Member of:")
for g in conn.getGroupsMemberOf():
    print("ID:", g.getName(), " Name:", g.getId())

From here onwards, I recommend having a tab with your jupyter notebook where you will be typing commands, and a second tab with your OMERO web interface open and logged in, so you can immediately see the results of what you are doing. Feel free to pause at any point and explore what is going on - taking it slow is encouraged! We will start by looking at how to create things in ezomero. 

The first thing we will do is create some hierarchy - we will add a project and a dataset to our OMERO server, where we can put images in a more organized way. 

In [ ]:
proj_id = ezomero.post_project(conn, "My New Project", "project description")
data_id = ezomero.post_dataset(conn, "My New Dataset", proj_id)

Note that, for `post_dataset`, we pass a project ID as well - that will create the new dataset inside that project. This is a good time to go to your OMERO web interface and refresh it, to see if you have a new Project and a new Dataset.

Now, we will add an image to that dataset. You have (hopefully) gone through the import process through OMERO.insight at this point - that is the preferred way to add existing image files to OMERO. What we will show now is a way to create an imagine in your OMERO server directly from a Python script, so be warned that this image is directly stored in disk as pixel values, not as an image file! It might be a good option when saving small analysis results, but it should be avoided as your main way of adding images to a server. The import process is more reliable and actually captures all the metadata from a file format!

Will all that said, here's what creating a new image directly looks like. We create a random `numpy` array of 8-bit integers (maximum value 256), with shape (200, 200, 20, 3, 1) for x, y, z, c, t, respectively. Then, we pass that array to `ezomero.post_image` as the pixels of our new image:

In [ ]:
import numpy as np
pixels = np.random.randint(256, size=(200, 200, 20, 3, 1), dtype=np.uint8)
im_id = ezomero.post_image(conn, pixels, "test", dataset_id=data_id)

Note that we get back the image ID that was generated for this new image. Also, note that `post_image` assumes the dimensional ordering to be `xyzct`, but that can be overridden by adding the extra argument `dim_order` (as usual, check the documentation for details!).
If you go to your web interface now, you should see a new image inside the new dataset - it will look kinda like static because we used random values for the pixels, but you have successfully created a new image! Great.

Now, we can add a few annotations to the new things we have created. Let's try adding a few map annotations (i.e. key-value pairs) to our new project and to our new image.

For those, we will use the `post_map_annotation` function. It requires an object type ("Project", for example), an object ID (we have those stored!), a Python dictionary with the key-value pairs to be added and a namespace. The namespace can be anything you want, but note that if you want your annotations to be editable from the OMERO.web interface there is a specific namespace you need to use (`"openmicroscopy.org/omero/client/mapAnnotation"`).

In [ ]:
proj_kv_dict = {"test_key": "test_value", "is this a project?": "yes"}
img_kv_dict = {"test_key": "test_value", "is this a project?": "no"}
ns = "myinstitution/myproject/test"
pj_map_id = ezomero.post_map_annotation(conn, "Project", 
                                        proj_id, proj_kv_dict, ns)
img_map_id = ezomero.post_map_annotation(conn, "Image", 
                                        im_id, img_kv_dict, ns)

Don't forget to close your connection at the end of a session!

In [ ]:
conn.close()

These are some of the basic creation tools you can get with ezomero - there are a lot of things not covered here, such as creating screens and plates, ROIs and file annotations. Take your time to explore the ezomero documentation and discover all the functionalities by yourself, and as always, if you would like to see a specific funcionality added, feel free to reach out to us or open an issue on Github!

This is probably a good time for a break. You can stop here and take 10 mins to get some water, stretch and then come back energized for the next part!